# Main Notebook
This Notebook contains all the code from all the other ones:
- nothotsongs
- 100hotsongs
- spotipy_api_webscrapper
- clustering_songs_from_dataframes

## Importing the Libraries

In [ ]:
import pandas as pd
import numpy as np
import pprint
import sys
sys.path.insert(1, '/Users/Hector_Martin/Documents/Labs/music_recommender_project/config.py')
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
import functions
%matplotlib inline

## Getting the Not Hot Songs DataFrame
Let's obtain a DataFrame containing a list of worth +5000 songs from the past.

In [ ]:
pd.set_option('display.max_columns', None)
music = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/EvolutionPopUSA_MainData.csv')

### Since the DataFrame consists of + 17000 songs, we are going to work with a fraction of the it

In [ ]:
nothotsongs = music.sample(frac=0.3, replace=False, random_state=1)
nothotsongs = nothotsongs[['artist_name', 'track_name']]

### Renaming the columns to keep consistency with the 100 Hot Songs DataFrame.

In [ ]:
nothotsongs = nothotsongs.rename(columns={'artist_name':'artists',
                            'track_name':'songs', })
nothotsongs = nothotsongs[['songs', 'artists']]
nothotsongs

### Storing the DataFrame into a csv file

In [ ]:
nothotsongs.to_csv("data/nothotsongs.csv", index=False)

## Web Scrapping of 100 Hot Songs
Our goal is to scrape the current top 100 songs present at https://www.billboard.com/charts/hot-100 and their respective artists, put the information into a pandas dataframe, and save the dataframe in a csv file in the current folder.

In [ ]:
hot100_df = functions.hot100("https://www.billboard.com/charts/hot-100/")
hot100_df

## Web Scrapping with Spotipy API

In order to get all the audio feautures from all the songs we have, we are going to use this API. Our final goal will be to get 3 DataFrames containing the name of the songs, the artists and their respective audio features obtained from Spotify:
- **100 Hot Songs DataFrame:** DataFrame cointaining the hottest 100 current mainstream hits
- **Not Hot Songs DataFrame:** DataFrame containing an extensive list of songs from the past
- **All Songs DataFrame:** DataFrame containing the content of the previous 2 ones

### Reading the files to get the DataFrames:

In [ ]:
hotsongs = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/hot100.csv')
nothotsongs = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/nothotsongs.csv')

### Getting the list of hot songs:

In [ ]:
hot_songs_list = [song for song in hotsongs['songs']]

### Let's do the same with the Not Hot Songs:

In [ ]:
not_hot_songs_list = [song for song in nothotsongs['songs']]

### Getting the audio features based on the lists of songs

In [ ]:
hot_songs_af = functions.get_audio_features(hot_songs_list)
not_hot_songs_af = functions.get_audio_features(not_hot_songs_list)

### Function to concatenate the Audio features to the song DataFrames

In [ ]:
hotconcat_df = functions.add_audio_features(hotsongs, hot_songs_af, 'hot_songs')
nothotconcat_df = functions.add_audio_features(nothotsongs, hot_songs_af, 'not_hot_songs')

### Function to concatenate the Hot Songs DataFrame with the Not Hot Songs one

In [ ]:
allsongs_df = functions.concatallsongs(hotconcat_df,nothotconcat_df,'allsongs_df')

## Clustering the songs from dataframes

### Importing the libraries

In [ ]:
all_songs = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/allsongs_df.csv')

### Removing all the unnecessary audio features:
These are not actually audio features, just links and other kind of information that has nothing to do with audio qualities.

In [ ]:
all_songs_clean = all_songs.drop(['analysis_url', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type'],axis =1)

### Store this cleaned Dataframe in a csv file:

In [ ]:
all_songs_clean.to_csv("data/all_songs_clean.csv", index=False)

### Numerical and Categorical split:
- X_num will be for Numerical columns
- X_cat will be for Categorical ones

In [ ]:
X_num = all_songs_clean.drop(['songs', 'artists'], axis =1)

In [ ]:
X_cat = all_songs[['songs', 'artists']]

### Scaling the features

In [ ]:
scaler = pickle.load(open("encoders/onehotencoder.pkl","rb"))
X_scaled = scaler.transform(X_num)
X_scaled_df = pd.DataFrame(X_scaled, columns = X_num.columns)
print('Data before the transformation')
print('------------------------------')
display(X_num.head())
print()
print('Data after the transformation')
print('------------------------------')
display(X_scaled_df.head())

### Training Models with different K values to assess which offers the best performance:

In [ ]:
functions.k_means_trainer(X_scaled_df)